//References
https://towardsdatascience.com/decision-tree-and-random-forest-from-scratch-4c12b351fe5e
https://www.kaggle.com/code/yashashwinidixit/decision-tree-from-scratch-in-python
https://towardsdatascience.com/adaboost-from-scratch-37a936da3d50

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import numpy as np
def calculateAccuracy(y_true, outputLabels):
    calculateAccuracy = np.sum(y_true == outputLabels) / len(y_true)
    return calculateAccuracy

In [ ]:
import pandas as pd
dataSet = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/sxb0600/train.csv")
dataSet.drop('PassengerId',axis=1,inplace=True)
dataSet.drop('Name',axis=1,inplace=True)
dataSet.drop('Ticket',axis=1,inplace=True)
dataSet.drop('Cabin',axis=1,inplace=True)
dataSet['Age'].fillna(dataSet['Age'].median(),inplace=True)
dataSet['Embarked'].fillna('S',inplace=True)
dataSet['Sex_Male'] = 1
dataSet['Sex_Female'] = 1 
for x in range(len(dataSet)):
    Temp_Sex = dataSet['Sex'][x]
    if Temp_Sex == "male":
        dataSet['Sex_Female'][x] = 0 
    elif Temp_Sex == "female":
        dataSet['Sex_Male'][x] = 0
dataSet.drop('Sex',axis=1,inplace=True)
dataSet['Embarked_S'] = 1
dataSet['Embarked_C'] = 1
dataSet['Embarked_Q'] = 1
for x in range(len(dataSet)):
    Temp_Embarked = dataSet['Embarked'][x]
    if Temp_Embarked == "S":
        dataSet['Embarked_C'][x] = 0
        dataSet['Embarked_Q'][x] = 0
    elif Temp_Embarked == "C":
        dataSet['Embarked_S'][x] = 0
        dataSet['Embarked_Q'][x] = 0
    elif Temp_Embarked == "Q":
        dataSet['Embarked_S'][x] = 0
        dataSet['Embarked_C'][x] = 0
dataSet.drop('Embarked',axis=1,inplace=True)
Train_Features = dataSet.iloc[:,1:].values
Train_Label = dataSet['Survived'].values
from sklearn.model_selection import train_test_split
featureTrain, featureTest, labelTrain, labelTest = train_test_split(Train_Features, Train_Label, test_size=0.5,random_state=123)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyW

In [ ]:
class TreeNode:
    def __init__( self, xfeature=None, minThreshold=None, leftChild=None, rightChild=None, *, info=None ):
        self.xfeature = xfeature
        self.minThreshold = minThreshold
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.info = info
    def isEndNode(self):
        return self.info is not None

In [ ]:
from collections import Counter
class DecisionTreeClassifier:
    def __init__(self, sampleSplitMinimum=5, maxDepthofTree=10, minSampleEndNode=None, criterion='gini'):
        self.default=None
        self.sampleSplitMinimum = sampleSplitMinimum
        self.maxDepthofTree = maxDepthofTree
        self.minSampleEndNode = minSampleEndNode
        self.root = self.default
    def train(self, xFeatures, yFeatures):
        self.minSampleEndNode = xFeatures.shape[1] if not self.minSampleEndNode else min(self.minSampleEndNode, xFeatures.shape[1])
        self.root = self.makeTree(xFeatures, yFeatures)
    def predict(self, xFeatures):
        return np.array([self.printTree(x, self.root) for x in xFeatures])
    def makeTree(self, xFeatures, yFeatures, height=0):
        uniqueLabels = len(np.unique(yFeatures))
        if ( height >= self.maxDepthofTree or uniqueLabels == 1 or xFeatures.shape[0] < self.sampleSplitMinimum ):
            counter = Counter(yFeatures)
            mostMatching = counter.most_common(1)[0][0] if counter!= Counter() else 0
            endLeafNodeValue = mostMatching
            return TreeNode(info=endLeafNodeValue)
        featIndex = np.random.choice(xFeatures.shape[1], self.minSampleEndNode, replace=False)
        maximumGain = -1
        splitIndex, splitThreshold = self.default, self.default
        for featID in featIndex:
            featureCol = xFeatures[:, featID]
            thrshold = np.unique(featureCol)
            for minThreshold in thrshold:
                gain = self.infoGain(yFeatures, featureCol, minThreshold)
                if gain > maximumGain:
                    maximumGain = gain
                    splitIndex = featID
                    splitThreshold = minThreshold
        maxFeat, maxThresh = splitIndex, splitThreshold
        leftChildIdx, rightChildIdx = self.seperate(xFeatures[:, maxFeat], maxThresh)
        leftChild = self.makeTree(xFeatures[leftChildIdx, :], yFeatures[leftChildIdx], height + 1)
        rightChild = self.makeTree(xFeatures[rightChildIdx, :], yFeatures[rightChildIdx], height + 1)
        return TreeNode(maxFeat, maxThresh, leftChild, rightChild)
    def infoGain(self, yFeatures, featureCol, minThreshold):
        ancestorEntropy = -np.sum([p * np.log2(p) for p in np.bincount(yFeatures) / len(yFeatures) if p > 0])
        leftChildIdx, rightChildIdx = self.seperate(featureCol, minThreshold)
        if len(leftChildIdx) == 0 or len(rightChildIdx) == 0:
            return 0
        n = len(yFeatures)
        entropyLeftNode = -np.sum([p * np.log2(p) for p in np.bincount(yFeatures[leftChildIdx]) / len(yFeatures[leftChildIdx]) if p > 0])
        entropyRightNode = -np.sum([p * np.log2(p) for p in np.bincount(yFeatures[rightChildIdx]) / len(yFeatures[rightChildIdx]) if p > 0])
        informationGainCollector = ancestorEntropy - ((len(leftChildIdx) / n) * entropyLeftNode + (len(rightChildIdx) / n) * entropyRightNode)
        return informationGainCollector
    def seperate(self, featureCol, splitThreshold):
        leftChildIdx = np.argwhere(featureCol <= splitThreshold).flatten()
        rightChildIdx = np.argwhere(featureCol > splitThreshold).flatten()
        return leftChildIdx, rightChildIdx
    def printTree(self, x, TreeNode):
        if TreeNode.isEndNode():
            return TreeNode.info
        if x[TreeNode.xfeature] <= TreeNode.minThreshold:
            return self.printTree(x, TreeNode.leftChild)
        return self.printTree(x, TreeNode.rightChild)

In [ ]:
from scipy import stats
class RandomForestClassifier:
    def train(self, xFeatures, yFeatures, minNumberofTree=7, minFeaturesofTree=None, maxDepthofTree=20, maxFeaturesofTree=None, bootstrapofTree=True, random_state=None):
        self.default=None
        self.minNumberofTree = minNumberofTree
        self.minFeaturesofTree = minFeaturesofTree
        self.maxDepthofTree = maxDepthofTree
        self.maxFeaturesofTree = maxFeaturesofTree
        self.bootstrapofTree = bootstrapofTree
        self.random_state = random_state
        self.decisionTreeList = []
        if len(self.decisionTreeList) > 0:
            self.decisionTreeList = []
        if isinstance(xFeatures, pd.core.frame.DataFrame):
            xFeatures = xFeatures.values
        if isinstance(yFeatures, pd.core.series.Series):
            yFeatures = yFeatures.values
        maxNumofBuilt = 0
        while maxNumofBuilt < self.minNumberofTree:
            classifierDTModel = DecisionTreeClassifier( maxDepthofTree=self.maxDepthofTree, minSampleEndNode=self.maxFeaturesofTree )
            numberofRows, _ = xFeatures.shape
            if self.minFeaturesofTree is self.default:
                sizeofSample = numberofRows
            else:
                sizeofSample = int(numberofRows*self.minFeaturesofTree)
            random_state = self.random_state
            np.random.seed(random_state)
            choices = np.random.choice(a=numberofRows, size=sizeofSample, replace=self.bootstrapofTree)
            classifierDTModel.train(xFeatures[choices],  yFeatures[choices])
            self.decisionTreeList.append(classifierDTModel)
            maxNumofBuilt += 1
            if self.random_state is not self.default:
                self.random_state += 1
    def predict(self, xFeatures):
        yFeatures = []
        for ithTree in self.decisionTreeList:
            yFeatures.append(ithTree.predict(xFeatures))
        yFeatures = np.swapaxes(yFeatures, axis1=0, axis2=1)
        modelOutputClasses = stats.mode(yFeatures,axis=1)[0].reshape(-1)
        return modelOutputClasses

In [ ]:
class AdaBoost:
    def train(self, xFeatures, yFeatures, mValue = 1000, LearnRateV=0.001, ValLearner = None):
        self.default=None
        self.noOfAlphas = []
        self.GMList = []
        self.mValue = mValue
        self.noOfAlphas = [] 
        mAlphaValue = 0
        for idxM in range(0, mValue):
            wiValue = (np.ones(len(yFeatures)) * 1 / len(yFeatures)) if idxM == 0 else wiValue * np.exp(mAlphaValue * (np.not_equal(yFeatures, outputLabels)).astype(int))
            gmValue = ValLearner(maxDepthofTree=1)    
            gmValue.train(xFeatures, yFeatures)
            outputLabels = gmValue.predict(xFeatures)
            self.GMList.append(gmValue) 
            mErrorValue = (sum(wiValue * (np.not_equal(yFeatures, outputLabels)).astype(int)))/sum(wiValue)
            mAlphaValue = LearnRateV*np.log((1 - mErrorValue) / mErrorValue)
            self.noOfAlphas.append(mAlphaValue)
    def predict(self, xFeatures):
        weakPredict = pd.DataFrame(index = range(len(xFeatures)), columns = range(self.mValue)) 
        for idxM in range(self.mValue):
            yPredmValue = self.GMList[idxM].predict(xFeatures) * self.noOfAlphas[idxM]
            weakPredict.iloc[:,idxM] = yPredmValue
        outputLabels = (1 * np.sign(weakPredict.T.sum())).astype(int)
        return outputLabels

In [ ]:
classifierDTModel = DecisionTreeClassifier(maxDepthofTree=15)
classifierDTModel.train(featureTrain, labelTrain)
outputLabels = classifierDTModel.predict(featureTest)
modelAccuracy = round(calculateAccuracy(labelTest, outputLabels), 2)
print("Total Accuracy Decision Tree :", modelAccuracy)

Total Accuracy Decision Tree : 0.78


In [ ]:
classifierDTModel = RandomForestClassifier()
classifierDTModel.train(featureTrain, labelTrain, minNumberofTree=30)
outputLabels = classifierDTModel.predict(featureTest)
modelAccuracy = round(calculateAccuracy(labelTest, outputLabels), 2)
print("Total Accuracy Random Forest:", modelAccuracy)

Total Accuracy Random Forest: 0.82


In [ ]:
classifierDTModel = AdaBoost()
classifierDTModel.train(featureTrain, labelTrain, mValue = 10,  LearnRateV=2, ValLearner=DecisionTreeClassifier)
outputLabels = classifierDTModel.predict(featureTest)
modelAccuracy = round(calculateAccuracy(labelTest, outputLabels), 2)
print("Total Accuracy Ada Boost :", modelAccuracy)

Total Accuracy Ada Boost : 0.55
